<a href="https://colab.research.google.com/github/mplaza27/googleScraper/blob/main/googleScrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install googlemaps
!pip install pandas
import googlemaps
import pandas as pd
import time
import requests
import json
from datetime import timedelta
from google.colab import drive
import os
import shutil

In [ ]:
def googleSearchInfo(key, para, location, dist, number_of_businesses, stop_time):
    START_TIME = time.time()


    #meters to miles
    def miles_to_meters(miles):
      try:
        return miles*1_609.344
      except:
        return 0


    map_client = googlemaps.Client(key)



    business_list = []

    #first finds
    response = map_client.places_nearby(
        location=location,
        keyword=para,
        radius=miles_to_meters(dist)
    )

    #iterating next finds
    business_list.extend(response.get('results'))
    next_page_token = response.get('next_page_token')


    loop_start_time = time.time()

    while next_page_token:
          curr_time = time.time() - loop_start_time
          if(stop_time is not None):
            time.sleep(1.5)
          response = map_client.places_nearby(
            location=location,
            keyword=para,
            radius=miles_to_meters(dist)
            )
          if (curr_time < 0.001):
            print("curr loop time: "+ str(curr_time*0)[:4])
          else:
            print("curr loop time: " + str(curr_time)[:4])
          if(number_of_businesses is not None and len(business_list) > number_of_businesses):
            break
          if(stop_time is not None and curr_time > stop_time):
            break

          business_list.extend(response.get('results'))
          next_page_token = response.get('next_page_token')


    #building dataframe
    df_partial = pd.DataFrame(business_list)
    df_partial['url'] = 'https://www.google.com/maps/place/?q=place_id:' + df_partial['place_id']
    df = df_partial.fillna("null")
    print(f"total run time: {time.time()- START_TIME}")
    return business_list, df

In [ ]:
def contactInfo(key, l1, fields):
    output = []
    headers = {}
    payload = {}

    for i in l1:
      #going through place_ids found earlier
      url = "https://maps.googleapis.com/maps/api/place/details/json?place_id=" + i['place_id'] + "&fields=" + fields + "&key=" + key
      response = requests.request("GET", url, headers=headers, data=payload)

      #making dictionaries (creating correct format)
      dictionary = json.loads(response.text)
      result = json.dumps(dictionary['result'])
      part_dict = json.loads(result)
      del dictionary['result']
      #counting dict building
      final_up = dictionary.copy()
      final_up.update(part_dict)

      #putting correct output
      output.append(final_up)


    #making dataframe correctly
    ds_partial = pd.DataFrame(output)


    print(f"Amount of buisnesses pulled: {len(ds_partial.keys()[0])}")


    return ds_partial

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

def uploadToCloud(filepath):

    prev = str(os.getcwd())
    filepath = prev + '/' + str(filepath)
    print(filepath)
    time.sleep(2)

    print(f"current path = {prev}")

    GDRIVE_PATH ='/content/drive/MyDrive/'
    os.chdir(GDRIVE_PATH)
    foldername = GDRIVE_PATH + 'googScrapeOutput/'

    if(not os.path.exists(foldername)):
      os.mkdir(foldername)
    os.chdir(foldername)

    curr = !pwd
    print(f"current path = {curr}")
    !ls

    time.sleep(2)
    shutil.copy(filepath, foldername)

    files = !ls

    print(f"files in current path are: {files}")
    os.chdir(prev)

    return prev, curr

In [ ]:
def organizeData(dataframe):
    columns = ['name', 'formatted_phone_number', 'formatted_address', 'website',
               'editorial_summary', 'types', 'price_level', 'rating', 'user_ratings_total',
               'dine_in', 'delivery', 'takeout', 'reservable', 'serves_breakfast', 'serves_brunch',
               'serves_dinner', 'serves_vegetarian_food', 'serves_wine', 'reviews', 'current_opening_hours', 'wheelchair_accessible_entrance'] #21 columns

    df_sorted = dataframe[columns]
    df = df_sorted.fillna("null")

    for index, row in df.iterrows():
        if(row['editorial_summary'] != 'null'):
          df['editorial_summary'][index] = row['editorial_summary']['overview']
        keys1 = ['author_name', 'author_url', 'rating', 'relative_time_description', 'text']
        if((row['reviews']) != 'null'):
          for i in range(0, len(row['reviews'])):
            df['reviews'][index][i] = {key: value for key, value in row['reviews'][i].items() if key in keys1}

        weekday_times = {}
        if(row['current_opening_hours'] != 'null'):

          for weekday in row['current_opening_hours']['weekday_text']:
              day, time = weekday.split(': ')
              time = time.replace('\u202f', ' ')
              time = time.replace('\u2009', '  ')
              weekday_times[day] = time
          df['current_opening_hours'][index] = weekday_times

    dc=df.sort_values(by='website',ascending=False)
    dc.insert(1, 'Initials', [None] * dc.shape[0])
    dc.insert(2, 'Date', [None] * dc.shape[0])
    dc.insert(3, 'Time', [None] * dc.shape[0])
    dc.insert(4, 'Duration', [None] * dc.shape[0])
    dc.insert(5, 'Key', [None] * dc.shape[0])
    dc.insert(6, 'Information', [None] * dc.shape[0])
    dc.insert(7, 'Tags', [None] * dc.shape[0])


    return dc

In [ ]:
def main():
    #API Key
    API_KEY = "Your API Key here"
    FIU = 'FIU', (25.753218375509416, -80.37607134523994)
    USABLE_FIELDS = ('serves_vegetarian_food%2Cwebsite%2Cserves_dinner%2Cserves_brunch'
    '%2Cprice_level%2Cwheelchair_accessible_entrance%2Cuser_ratings_total%2Ctypes%2Ctakeout'
    '%2Cserves_wine%2Cserves_breakfast%2Creviews%2Creservable%2Cname%2Crating%2C'
    'formatted_phone_number%2Cformatted_address%2Ceditorial_summary%2Cdine_in%2Cdelivery%2C'
    'current_opening_hours%2Cbusiness_status')


    para = "restaraunts"
    fields = USABLE_FIELDS
    name, location = FIU


                                                                              #miles        #batches of 20          #should be from 5-12 seconds
    l, df = googleSearchInfo(key = API_KEY, para = para, location = location, dist = 15, number_of_businesses = 22, stop_time = None) #prints 40
    ds = contactInfo(key = API_KEY, l1 = l, fields = fields)

    n = len(ds[ds.keys()[0]])

    dfinaldata = organizeData(ds)

    filename = f'{para}_{name}_{n}.xlsx'

    dfinaldata.to_excel(filename, index=False)
    uploadToCloud(filename)

main()